# **Householder Notebook**

@eti-calde

---

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

## **1.1 Householder Reflection**

**Definition**
Given a unit vector $u$ defining a hyperplane perpendicular to ir, a Householder reflection over a vector $x$ is defined as:

$$
H_{\mathbf{u}}(x) = x-2 \langle \mathbf{x},\mathbf{u}\rangle \mathbf{u}
$$

Is nothing more that the vector reflected over the hyperplane. This can be expressed in matrix form as follows:

$$
H =(I-2\mathbf{u}^T\mathbf{u})
$$

---

## **1.2 let’s see the transformation in code!!**

In [8]:
def get_householder_matrix(v):
    """
    Calculates the Householder reflection matrix for a vector v
    
    Parameters
    ----------
    v : ndarray (n,), input vector.
    Returns
    -------
    H : ndarray (n,n), Householder reflection matrix.
    """
    v = v.reshape(-1, 1)
    I = np.eye(v.shape[0])
    return I - 2 * (v @ v.T) / (v.T @ v)

In [9]:
cmap = plt.get_cmap("magma")
C_V1   = cmap(0.15)  # x
C_V2   = cmap(0.35)  # u
C_PLN  = cmap(0.85)  # hyperplane
C_REF  = cmap(0.60)  # H(x)

def plot_householder(theta1_deg=30.0, theta2_deg=120.0, r1=1.5):
    # Parameters
    t1 = np.deg2rad(theta1_deg)
    t2 = np.deg2rad(theta2_deg)
    unitari_rad = 1.0

    # Vectors
    x = np.array([r1*np.cos(t1), r1*np.sin(t1)])
    u = np.array([unitari_rad*np.cos(t2), unitari_rad*np.sin(t2)])

    # Hyperplane
    n_perp = np.array([-u[1], u[0]])

    # Householder
    H = get_householder_matrix(u)
    x_householder = H @ x
    print("x:", x)
    print("x' (Householder):", x_householder)
    # ===================================================================
    # PLOT
    plt.figure()

    # Vectors
    plt.quiver(0, 0, x[0], x[1], color=C_V1, scale=1, scale_units='xy', angles='xy', label='x')
    plt.quiver(0, 0, u[0], u[1], color=C_V2, scale=1, scale_units='xy', angles='xy', label='u (unitario)')
    plt.quiver(0, 0, x_householder[0], x_householder[1],
               color=C_REF, scale=1, scale_units='xy', angles='xy', label='H(x)')
    # Hyperplane
    L = 2.2
    pts = np.vstack([-L*n_perp, L*n_perp])
    plt.plot(pts[:,0], pts[:,1], linestyle='--', color=C_PLN, label='Hip. u·x = 0')

    # Details
    lim = 2.2
    plt.xlim(-lim, lim); plt.ylim(-lim, lim)
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title("Householder reflection of x")
    plt.show()

angle1 = FloatSlider(value=30.0,  min=0.0, max=360.0, step=1.0, description='Ángulo x (°)')
angle2 = FloatSlider(value=120.0, min=0.0, max=360.0, step=1.0, description='Ángulo u (°)')
len_r1 = FloatSlider(value=1.5,   min=0.0, max=2.0,   step=0.1, description='||x||')

interact(plot_householder, theta1_deg=angle1, theta2_deg=angle2, r1=len_r1)


interactive(children=(FloatSlider(value=30.0, description='Ángulo x (°)', max=360.0, step=1.0), FloatSlider(va…

<function __main__.plot_householder(theta1_deg=30.0, theta2_deg=120.0, r1=1.5)>

Now let's apply the Householder Reflection to get the QR factorization of a matrix $A$.

---

## **2.3 QR Factorization**

In [10]:
def get_Q_Matrix(householder_vectors):
    """
    Calculates Q matrix of QR factorization using Householder reflections.

    Parameters
    ----------
    householder_vectors : ndarray (n,n), Householder reflection vectors.
    each i-th column represents a Householder vector for the i-th step of the QR factorization.

    Returns
    -------
    Q : ndarray (n,n), Orthogonal matrix.
    """
    n = householder_vectors.shape[0]
    Q = np.eye(n)
    for i in range(n):
        u = householder_vectors[:n-i, i]
        H_tilde = get_householder_matrix(u)
        H_full = np.eye(n)
        H_full[i:, i:] = H_tilde
        Q = Q @ H_full
    return Q

def QR_Householder(A):
    """
    QR factorization using Householder reflections.

    Parameters
    ----------
    A : ndarray (n,n), input matrix.
    Returns
    -------
    Q,R : ndarray (n,n), Orthogonal matrix and upper triangular matrix.
    """
    if (A.shape[1] != A.shape[0]):
        print("Matrix is not square")
        return None, None
    
    n = A.shape[0]
    # We need to work with submatrices of A
    # and we don't want to modify the original Matrix
    A = A.copy()

    # Each Householder vector is 1 dimension smaller than the previous one,
    # So we use just the first n-i rows on each iteration 
    householder_vectors = np.zeros((n, n))

    for i in range(n):
        sub_A = A[i:, i:]
        # Get Column i of A
        col_i_of_A = sub_A[:,0]
        col_i_of_A = col_i_of_A.reshape(-1,1)

        # Compute the Householder vector
        alpha = -np.sign(col_i_of_A[0,0]) * np.linalg.norm(col_i_of_A)
        e1 = np.eye(col_i_of_A.shape[0])[:,0].reshape(-1,1)
        u = col_i_of_A - alpha * e1
        u = u / np.linalg.norm(u)
        # print(f"u en la iteracion {i}, {u.shape} =\n", u)

        # Save the Householder vector
        householder_vectors[0:n-i,i] = u.ravel()
        # print(f"holder_vectors en la iteracion {i} =\n", holder_vectors)
        
        # FIX RESPECTO A LA VERSIÓN DE LA CALSE ANTERIOR.
        A[i:, i:] -= 2 * np.outer(u, u.T @ A[i:, i:])

    Q = get_Q_Matrix(householder_vectors)
    R = np.triu(A)
    # R = A
    return Q,R

In [11]:
size = 4
A = np.random.rand(size,size) + np.identity(size)*2
print("Matriz A\n",A)
Q,R = QR_Householder(A)
# print("uni?:\n", Q.T@Q)
compare_numpy = True
if (size <= 5):
    # print("A =\n", A)
    print("R:\n", R)
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("R (numpy):\n", R_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))
else:
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))

Matriz A
 [[2.87742297 0.15580567 0.08816251 0.88475573]
 [0.85775538 2.78490655 0.73079871 0.73045657]
 [0.26102913 0.03439986 2.84714746 0.62531842]
 [0.12169719 0.78915069 0.71503131 2.50595675]]
R:
 [[-3.01633116 -0.97539163 -0.56715692 -1.20695094]
 [ 0.         -2.72993282 -0.79047775 -1.09670894]
 [ 0.          0.         -2.86580456 -0.91861662]
 [ 0.          0.          0.         -2.11750898]]
Reconstrucción ||A - Q R|| = 8.886119947416683e-16
Comparando con numpy...
R (numpy):
 [[-3.01633116 -0.97539163 -0.56715692 -1.20695094]
 [ 0.         -2.72993282 -0.79047775 -1.09670894]
 [ 0.          0.         -2.86580456 -0.91861662]
 [ 0.          0.          0.          2.11750898]]
Reconstrucción ||A - Q R|| = 1.828829704940152e-15


---
## **2.4 Arnoldi Householder**

### Idea Principal

El resultado del algoritmo de Arnoldi se basa en la reducción parcial de $A$ a una forma Hesenberg:
$$
A V_k = V_{k+1} \tilde{H}_k \tag{1}
$$

Recordemos que $V_k$ es el espacio que deseamos ortonormalizar qu esta compuesto por los vectores del sub espacio de Krylov:
$$
V_k = \{r_0, A r_0, A^2 r_0, \dots, A^{k-1}r_0\}
$$

Ahora notamos que el lado izquierdo de la ecuación **(1)** $A V_k$ podemos escribir la multiplicación así:
$$
A V_k = \{Ar_0, A^2  r_0, A^3 r_0, \dots, A^{k}r_0\}
$$

Lo cual es muy cercano a $V_{k+1}$, pero nos faltaría agregar la primera columna al espacio que es $r_0$, llegando a:
$$
[r_0 A V_k] = V_{k+1} R_k \tag{2}
$$

Donde:
$$
R_k =
\begin{bmatrix}
\beta  & h_{1,1} & \dots  & h_{1,k} \\
0      & h_{2,1} & \dots  & h_{2,k} \\
\vdots & \vdots & \ddots & \vdots \\
0      & 0      & \dots  & h_{k+1,k}
\end{bmatrix}
$$

El objetivo de hacer esto es que ahora en la ecucación **(2)** tenemos una matriz del lado izquierdo $[r_0 A V_k]$ y del lado derecho una matriz ortogonal $V_{k+1}$ y una matriz triangular superior $R_k$ de decir podemos llegar al lado derecho aplicando QR al lado izquierdo.

### Detalles

Para aplicar Arnoldi en GMRes no se parte de la base completa $V_k$, mas bien se construye en cada iteración un nuevo vector de la base, por lo cual surge la duda ¿cómo se obtiene el vector $v_i$?

1. El plan es hacer factorización QR sobre $[r_0 A V_k]$:
$$
[r_0 A V_k] = Q R
$$

donde $Q = V_{k+1}$ porque la construcción viene de la reducción parcial de Hessenberg y por lo aprendido en QR Householder, sabemos que:
$$
V_{k+1} = Q = P_1 P_2 \dots P_{k+1}
$$

Las columnas de la matriz $V_{k+1}$ son los $v_i$ que andamos buscando y ahora es sencillo que que si queremos obenter la i-ésima columna podemos:

$$
v_i = V_{k+1} e_i = (P1 P2 \dots P_{k+1}) e_i
$$

Es más aún, dada la estructura de las matrices $P$, recordad que cada una es una identidad con una submatriz dentro, por lo tanto:

$$
v_i = P_1 P_2 \dots P_j e_j
$$





In [12]:
size = 5
A = np.random.rand(size, size)
e3 = np.zeros(size)
e3[2] = 1
print(A)
print("\n",A @ e3)

[[0.82563423 0.61792168 0.69668293 0.36324593 0.20368468]
 [0.67801809 0.22610812 0.30042735 0.07649486 0.12856138]
 [0.15002693 0.15064484 0.89162775 0.16935691 0.08020385]
 [0.32795632 0.00549266 0.27327301 0.28537406 0.35351595]
 [0.49941065 0.28411966 0.48945144 0.46648805 0.03945705]]

 [0.69668293 0.30042735 0.89162775 0.27327301 0.48945144]


In [31]:
def get_householder_vector(x):
    if np.linalg.norm(x) < 1e-16:
        return np.zeros_like(x)
    
    alpha = np.linalg.norm(x)
    e1 = np.zeros_like(x)
    e1[0] = 1.0
    
    # El signo garantiza la estabilidad numérica evitando la resta de números cercanos
    v = x + np.copysign(alpha, x[0]) * e1
    v /= np.linalg.norm(v)
    return v

# =============== TEST ===============

# rand_vec = np.random.rand(3)
# norm = np.linalg.norm(rand_vec)
# householder_vec = get_householder_vector(rand_vec)
# apply_householder = rand_vec - 2 * householder_vec * (householder_vec.T @ rand_vec)
# e1 = np.zeros(3)
# e1[0] = 1
# print("original vec:\n", rand_vec)
# print("apply householder vec:\n", apply_householder)
# print("|rand vec| * e1:\n", norm * e1)

# =====================================


In [32]:
def apply_householder_sequence(x, householder_vecs, reverse=False):
    vec = x.copy()
    iterator = reversed(householder_vecs) if reverse else householder_vecs
    for w in iterator:
        vec = vec - 2 * w * (w.T @ vec)     
    return vec

In [33]:
verbose = True
print(f"print only if verbose is true, verbose = {verbose}") if verbose else None


print only if verbose is true, verbose = True


In [34]:
def householder_arnoldi(A, r0, m, verbose=False):
    n = A.shape[0]
    V = np.zeros((n, m + 1))
    H_bar = np.zeros((m + 1, m))
    householder_vecs = []
    z = r0.copy() # Vectors to transform to form H_bar

    # Main loop from j=1 to m+1
    for j in range(1, m + 2):
        
        print(f"\n =============== ITERARTION j={j} =============== ")  if verbose else None
        
        print(f"\n =================== STEP 1 =================== ")    if verbose else None
        print(f"1. Get householder vector w_{j} hat send z to |z|e1")   if verbose else None
        
        sub_z       = z[j - 1:]
        v           = get_householder_vector(sub_z)        
        w_j         = np.zeros(n)
        w_j[j - 1:] = v
        householder_vecs.append(w_j)
        
        print(f"2. Householder Vector w_{j}:\n {w_j}")                   if verbose else None
        print(f"\n ================================================ \n") if verbose else None


        print(f"\n =================== STEP 2 =================== ")     if verbose else None
        print("1. Apply the householder transformation w_{j} to z ")     if verbose else None
        print("We do not update H_bar on the first iteration")           if verbose and j==1 else None

        h_full = z - 2 * w_j * (w_j.T @ z)
        if j > 1 and j <= m + 1:
            H_bar[:, j - 2] = h_full[:m + 1]
            print(f"2. H_bar updated (column {j - 2}):\n", H_bar, "\n")  if verbose else None
        
        print(f"\n ================================================ \n") if verbose else None
        
        
        print(f"\n =================== STEP 3 =================== ")     if verbose else None
        print("1. Construct the new vector of V applying the householder sequence")   
        e_j = np.zeros(n)
        e_j[j - 1] = 1.0
        v_j = apply_householder_sequence(e_j, householder_vecs, reverse=True)
        V[:, j - 1] = v_j
        
        print(f"2. new v_{j} = \n{v_j}")
        print(f"3. add v_{j} a V[:, {j - 1}] \n{V}")
        print(f"\n ================================================ \n")


        if j <= m:
            print(f"\n =================== STEP 4 =================== ")  if verbose else None
            print("1. Prepare z for the next iteration")
            Av = A @ v_j
            z = apply_householder_sequence(Av, householder_vecs, reverse=False)
            print(f"new vector z_{j+1} = A v_{j} and then apply old reflections  = \n{z}\n")
            print(f"\n ================================================ \n")

    V_m = V[:, :m]
    V_m_plus_1 = V

    if verbose:
        print(f"\n =================== EXTRA CHECKS =================== ")
        print("1. Check the orthonormality of V_m: ")
        print("| I - V_m^T @ V_m | :", np.linalg.norm(np.identity(m)- (V_m.T @ V_m)), "\n")
        LHS = A @ V_m
        RHS = V_m_plus_1 @ H_bar
        error = np.linalg.norm(LHS - RHS)
        print("2. Verify Arnoldi's fundamental identity ")
        print(print(f"\nError ||A*V_m - V_{{m+1}}*H_bar||: {error:.2e}"))

    return V_m, V_m_plus_1, H_bar


np.random.seed(0)
n = 3
A = np.random.rand(n, n)
r0 = np.random.rand(n)
r0 = r0 / np.linalg.norm(r0)
m = 2

V_m, V_m_plus_1, H_bar = householder_arnoldi(A, r0, m, verbose=True)




 =============== ITERARTION j=1 =============== 

 =================== STEP 1 =================== 
1. Get householder vector w_1 hat send z to |z|e1
2. Householder Vector w_1:
 [0.8287229  0.46537184 0.31088167]



 =================== STEP 2 =================== 
1. Apply the householder transformation w_{j} to z 
We do not update H_bar on the first iteration



 =================== STEP 3 =================== 
1. Construct the new vector of V applying the householder sequence
2. new v_1 = 
[-0.37356329 -0.77132859 -0.51526951]
3. add v_1 a V[:, 0] 
[[-0.37356329  0.          0.        ]
 [-0.77132859  0.          0.        ]
 [-0.51526951  0.          0.        ]]



 =================== STEP 4 =================== 
1. Prepare z for the next iteration
new vector z_2 = A v_1 and then apply old reflections  = 
[ 1.7589579   0.72392956 -0.28765818]




 =============== ITERARTION j=2 =============== 

 =================== STEP 1 =================== 
1. Get householder vector w_2 hat send 

---
# Gmres Householder \#1

- La idea es re-utilizar el mismo código de Arnoldi Householder, pero implementando GMRes, es decir, resolviendo el problema de minimización:
$$
y_m = \min_{y \in \mathbb{R}^m} \|\beta e_1 - \tilde{H}y \|
$$

- Para encontrar la solución al sistema, en este primer enfoque no se utilizaran tranformaciones de Givens para resolver el problema de minimización

In [14]:
import numpy as np

def gmres_householder_lstsq_corregido(A, b, m, tol=1e-10, verbose=False):
    n = A.shape[0]
    
    # --- INICIALIZACIÓN DE GMRES ---
    x0 = np.zeros(n)
    r0 = b - A @ x0
    beta = np.linalg.norm(r0)
    
    residuals_history = [beta]
    
    if beta < tol:
        return x0, residuals_history

    # --- FASE 1: PROCESO DE HOUSEHOLDER-ARNOLDI ---
    V = np.zeros((n, m + 1))
    H_bar = np.zeros((m + 1, m))
    w_vectors = []
    z = r0.copy()
    
    rhs_scalar = 0.0 ### CAMBIO 1: Variable para guardar el escalar correcto para el rhs

    for j in range(1, m + 2):
        if verbose:
            print(f"\n =============== Iteración j={j} =============== ")
        
        sub_z = z[j - 1:]
        alpha = np.linalg.norm(sub_z)
        if alpha == 0:
            v = np.zeros_like(sub_z)
        else:
            e1_sub = np.zeros_like(sub_z)
            e1_sub[0] = 1.0
            v = sub_z + np.copysign(alpha, sub_z[0]) * e1_sub
            v /= np.linalg.norm(v)

        w_j = np.zeros(n)
        w_j[j - 1:] = v
        w_vectors.append(w_j)
        
        h_full = z - 2 * w_j * (w_j.T @ z)
        
        ### CAMBIO 1: Capturamos el escalar en la primera iteración
        if j == 1:
            rhs_scalar = h_full[0]
        
        if j > 1 and j <= m + 1:
            H_bar[:, j - 2] = h_full[:m + 1]

        v_j = np.zeros(n)
        v_j[j - 1] = 1.0
        for i in range(j, 0, -1):
            w_i = w_vectors[i - 1]
            v_j = v_j - 2 * w_i * (w_i.T @ v_j)
        V[:, j - 1] = v_j

        if j <= m:
            Av = A @ v_j
            z_next = Av.copy()
            for i in range(1, j + 1):
                w_i = w_vectors[i - 1]
                z_next = z_next - 2 * w_i * (w_i.T @ z_next)
            z = z_next

    # ### CAMBIO 2: Eliminamos la línea que corrompía la matriz V
    # V[:, m-1] = v_j  <-- ESTA LÍNEA SE HA ELIMINADO
    
    V_m = V[:, :m]
    
    # 1. Definir el lado derecho del sistema de mínimos cuadrados
    rhs = np.zeros(m + 1)
    rhs[0] = rhs_scalar ### CAMBIO 1: Usamos el escalar correcto
    
    # 2. Resolver el sistema para encontrar y
    y, _, _, _ = np.linalg.lstsq(H_bar, rhs, rcond=None)
    
    # 3. Calcular la solución final actualizando x0
    x_final = x0 + V_m @ y
    
    # Cálculo del historial de residuos para análisis
    for k in range(1, m):
        yk, _, _, _ = np.linalg.lstsq(H_bar[:k+1, :k], rhs[:k+1], rcond=None)
        xk = x0 + V[:, :k] @ yk
        residuals_history.append(np.linalg.norm(b - A @ xk))
    
    residuals_history.append(np.linalg.norm(b - A @ x_final))

    return x_final, residuals_history

# --- EJEMPLO DE USO ---
np.random.seed(42)
n = 10
A = np.random.rand(n, n) + np.identity(n) * 2
b = np.random.rand(n)
m = 8

# Ejecutar el solver corregido
x_sol, res_hist = gmres_householder_lstsq_corregido(A, b, m)

# Verificar la solución
final_residual_norm = np.linalg.norm(b - A @ x_sol)
print(f"Solución encontrada. Norma del residuo final: {final_residual_norm:.4e}")
print("\nHistorial de la norma del residuo (calculado a posteriori):")
for i, res in enumerate(res_hist):
    print(f"  Iteración {i}: {res:.4e}")

Solución encontrada. Norma del residuo final: 3.4400e-04

Historial de la norma del residuo (calculado a posteriori):
  Iteración 0: 1.5642e+00
  Iteración 1: 6.6860e-01
  Iteración 2: 2.5331e-01
  Iteración 3: 8.7207e-02
  Iteración 4: 3.8287e-02
  Iteración 5: 1.8015e-02
  Iteración 6: 3.3900e-03
  Iteración 7: 1.1215e-03
  Iteración 8: 3.4400e-04


---

# Gmres Householde \#2

Ahora la idea es aplicar rotaciones de Givens para ir transformando la matriz de Hessenberg en triangular superior, lo cual facilita el problema de minimización a resolver

In [15]:
import numpy as np

def gmres_householder_givens(A, b, x0, m, tol=1e-10, verbose=False):
    """
    Implementación del algoritmo GMRES utilizando transformaciones de Householder
    para el proceso de Arnoldi, como se describe en el "Algorithm 4.1".

    Args:
        A (np.ndarray): Matriz del sistema.
        b (np.ndarray): Vector del lado derecho.
        x0 (np.ndarray): Estimación inicial.
        m (int): Número máximo de iteraciones (dimensión del subespacio de Krylov).
        tol (float): Tolerancia para el criterio de parada.
        verbose (bool): Si es True, imprime información detallada en cada paso.

    Returns:
        tuple: La solución x y una lista con la norma de los residuos en cada iteración.
    """
    n = A.shape[0]

    # --- INICIALIZACIÓN DE GMRES ---
    r0 = b - A @ x0
    beta = np.linalg.norm(r0)
    
    # Lista para almacenar la norma de los residuos en cada iteración
    residuals = [beta]
    
    # Si la solución inicial es suficientemente buena, la devolvemos
    if beta < tol:
        return x0, residuals

    # Inicialización de variables para Arnoldi
    V = np.zeros((n, m + 1))
    H_bar = np.zeros((m + 1, m))
    w_vectors = []  # Almacena los vectores de Householder
    z = r0.copy()   # 'z' es el vector a transformar en cada paso
    
    # --- COMPONENTES PARA GIVENS ---
    # c y s almacenan los cosenos y senos de las rotaciones de Givens
    c = np.zeros(m)
    s = np.zeros(m)
    # g es el lado derecho del sistema de mínimos cuadrados, que se actualizará
    g = np.zeros(m + 1)
    g[0] = beta
    
    # --- BUCLE PRINCIPAL DE GMRES (combina Arnoldi, Givens y chequeo de residuo) ---
    for j in range(1, m + 1):
        if verbose:
            print(f"\n =============== Iteración j={j} =============== ")

        # === PASO 1: ARNOLDI CON HOUSEHOLDER (Tu código, casi sin cambios) ===
        # Cálculo del vector de Householder w_j
        sub_z = z[j - 1:]
        alpha = np.linalg.norm(sub_z)
        v = np.zeros_like(sub_z)
        if alpha > 1e-16: # Evitar división por cero
            e1_sub = np.zeros_like(sub_z)
            e1_sub[0] = 1.0
            v = sub_z + np.copysign(alpha, sub_z[0]) * e1_sub
            v /= np.linalg.norm(v)

        w_j = np.zeros(n)
        w_j[j - 1:] = v
        w_vectors.append(w_j)
        
        # Aplicación de la transformación a z para obtener la columna de H_bar
        h_col = z - 2 * w_j * (w_j.T @ z)
        
        # Guardamos la columna de Hessenberg
        H_bar[:j+1, j-1] = h_col[:j+1]

        # === PASO 2: APLICAR ROTACIONES DE GIVENS ANTERIORES ===
        # [cite: 124]
        # Aplicamos las rotaciones de Givens de las iteraciones 1 a j-1
        # a la nueva columna de H_bar.
        for i in range(j - 1):
            temp1 = c[i] * H_bar[i, j-1] + s[i] * H_bar[i+1, j-1]
            temp2 = -s[i] * H_bar[i, j-1] + c[i] * H_bar[i+1, j-1]
            H_bar[i, j-1] = temp1
            H_bar[i+1, j-1] = temp2

        # === PASO 3: CALCULAR Y APLICAR LA NUEVA ROTACIÓN DE GIVENS ===
        # [cite: 74-77]
        # El objetivo es eliminar el elemento H_bar[j, j-1]
        h_jj_1 = H_bar[j-1, j-1]
        h_j1_j = H_bar[j, j-1]
        
        norma = np.sqrt(h_jj_1**2 + h_j1_j**2)
        c[j-1] = h_jj_1 / norma
        s[j-1] = h_j1_j / norma
        
        # Aplicamos la nueva rotación a la matriz H_bar
        H_bar[j-1, j-1] = c[j-1] * H_bar[j-1, j-1] + s[j-1] * H_bar[j, j-1]
        H_bar[j, j-1] = 0.0 # Se anula por definición
        
        # Aplicamos la nueva rotación al vector g
        # 
        g[j] = -s[j-1] * g[j-1]
        g[j-1] = c[j-1] * g[j-1]
        
        # === PASO 4: CHEQUEO DE CONVERGENCIA ===
        # La norma del residuo es el valor absoluto del último elemento de g
        residual_norm = abs(g[j])
        residuals.append(residual_norm)
        if verbose:
            print(f"Norma del residuo en iteración {j}: {residual_norm:.4e}")
            
        if residual_norm < tol:
            print(f"Convergencia alcanzada en la iteración {j}.")
            m = j # Ajustamos m para el cálculo final de la solución
            break

        # === PASO 5: PREPARACIÓN PARA LA SIGUIENTE ITERACIÓN (ARNOLDI) ===
        # Reconstrucción de v_j
        v_j = np.zeros(n)
        v_j[j - 1] = 1.0
        for i in range(j, 0, -1):
            w_i = w_vectors[i - 1]
            v_j = v_j - 2 * w_i * (w_i.T @ v_j)
        V[:, j - 1] = v_j
        
        # Calcular z para la siguiente iteración
        if j < m + 1:
            Av = A @ v_j
            z_next = Av.copy()
            for i in range(1, j + 1):
                w_i = w_vectors[i - 1]
                z_next = z_next - 2 * w_i * (w_i.T @ z_next)
            z = z_next

    # --- CÁLCULO FINAL DE LA SOLUCIÓN ---
    # [cite: 103, 129]
    # 1. Reconstruir V_m final (necesitamos hasta v_m)
    for k in range(1, m + 1):
        v_k = np.zeros(n)
        v_k[k - 1] = 1.0
        for i in range(k, 0, -1):
            w_i = w_vectors[i - 1]
            v_k = v_k - 2 * w_i * (w_i.T @ v_k)
        V[:, k - 1] = v_k
    V_m = V[:, :m]
    
    # 2. Resolver el sistema triangular superior R*y = g_m
    R = H_bar[:m, :m]
    g_m = g[:m]
    y = np.linalg.solve(R, g_m)
    
    # 3. Actualizar la solución final
    x = x0 + V_m @ y
    
    return x, residuals

# --- EJEMPLO DE USO ---
np.random.seed(42)
n = 10
A = np.random.rand(n, n) + np.identity(n) * 2
b = np.random.rand(n)
x0 = np.zeros(n)
m = 8 # Máximo de iteraciones

# Ejecutar el solver
x_sol, residuals_history = gmres_householder_givens(A, b, x0, m, tol=1e-8, verbose=True)

# Verificar la solución
final_residual_norm = np.linalg.norm(b - A @ x_sol)
print(f"\nSolución encontrada. Norma del residuo final: {final_residual_norm:.4e}")
print("Historial de la norma del residuo:")
for i, res in enumerate(residuals_history):
    print(f"  Iteración {i}: {res:.4e}")


 =============== Iteración j=1 =============== 
Norma del residuo en iteración 1: 1.1102e-16
Convergencia alcanzada en la iteración 1.

Solución encontrada. Norma del residuo final: 4.3816e+00
Historial de la norma del residuo:
  Iteración 0: 1.5642e+00
  Iteración 1: 1.1102e-16
